# (Purpose) Batch effect correction (includes different algorithms and addressing specific batch types)

In [ ]:
# ---------------------- Batch Effect Algorithms --------------
BATCH_EFFECT_ALGORITHM_TO_APPLY = "limma"
# BATCH_EFFECT_ALGORITHM_TO_APPLY = "combat"  
# BATCH_EFFECT_ALGORITHM_TO_APPLY = "poibm" 

# ---------------------- Batch Effect Combinations --------------

APPLY_BATCH_EFFECT_ON_PROTOCOL = TRUE              
# APPLY_BATCH_EFFECT_ON_PROTOCOL = FALSE

APPLY_BATCH_EFFECT_ON_DISEASE = TRUE
# APPLY_BATCH_EFFECT_ON_DISEASE = FALSE

APPLY_BATCH_EFFECT_ON_CONSORTIUM = TRUE
# APPLY_BATCH_EFFECT_ON_CONSORTIUM = FALSE

# ---------------------- TCGA dataset ---------------------------              ** make sure input and outputs match!
# ---------------------------- input file------------------------
filename_tcga = "tcga_unscaled_unnormalized_nobatchcorrection__mockData.tsv"  
# filename_tcga = "tcga_unscaled_unnormalized_nobatchcorrection.tsv"      # real data0 (baseline)
# filename_tcga = "tcga_unscaled_qn_nobatchcorrection.tsv"                # real data2
# filename_tcga = "tcga_unscaled_qntarget_nobatchcorrection.tsv"          # real data4
# filename_tcga = "tcga_unscaled_fsqn_nobatchcorrection.tsv"              # real data6

# ---------------------------- output file-----------------------
# save_filename_tcga = "tcga_unscaled_unnormalized_batchcorrected.tsv"    # real data1
# save_filename_tcga = "tcga_unscaled_unnormalized_batchcorrected_protocol.tsv"    
# save_filename_tcga = "tcga_unscaled_unnormalized_batchcorrected_disease.tsv"    
# save_filename_tcga = "tcga_unscaled_unnormalized_batchcorrected_consortium.tsv"   
# save_filename_tcga = "tcga_unscaled_unnormalized_batchcorrected_protocoldisease.tsv"   
# save_filename_tcga = "tcga_unscaled_unnormalized_batchcorrected_protocolconsortium.tsv"   
# save_filename_tcga = "tcga_unscaled_unnormalized_batchcorrected_diseaseconsortium.tsv"   
# save_filename_tcga = "tcga_unscaled_qn_batchcorrected.tsv"              # real data3
# save_filename_tcga = "tcga_unscaled_qntarget_batchcorrected.tsv"        # real data5
# save_filename_tcga = "tcga_unscaled_fsqn_batchcorrected.tsv"            # real data7

WORKING_WITH_INDEPENDENT="GTEX"
# WORKING_WITH_INDEPENDENT="ICGC"
# WORKING_WITH_INDEPENDENT="GEO"
# WORKING_WITH_INDEPENDENT="ICGCGEO"

# ---------------------- GTEx dataset ---------------------------     
# ---------------------------- input file------------------------
filename_gtex = "gtex_unscaled_unnormalized_nobatchcorrection__mockData.tsv"  
# filename_gtex = "gtex_unscaled_unnormalized_nobatchcorrection.tsv"      # real data0 (baseline)
# filename_gtex = "gtex_unscaled_qn_nobatchcorrection.tsv"                # real data2
# filename_gtex = "gtex_unscaled_qntarget_nobatchcorrection.tsv"          # real data4
filename_gtex = "gtex_unscaled_fsqn_nobatchcorrection.tsv"              # real data6

# ---------------------------- output file-----------------------
# save_filename_gtex = "gtex_unscaled_unnormalized_batchcorrected.tsv"    # real data1
# save_filename_gtex = "gtex_unscaled_unnormalized_batchcorrected_protocol.tsv"   
# save_filename_gtex = "gtex_unscaled_unnormalized_batchcorrected_disease.tsv"    
# save_filename_gtex = "gtex_unscaled_unnormalized_batchcorrected_consortium.tsv"    
# save_filename_gtex = "gtex_unscaled_unnormalized_batchcorrected_protocoldisease.tsv"   
# save_filename_gtex = "gtex_unscaled_unnormalized_batchcorrected_protocolconsortium.tsv"   
# save_filename_gtex = "gtex_unscaled_unnormalized_batchcorrected_diseaseconsortium.tsv"   
# save_filename_gtex = "gtex_unscaled_qn_batchcorrected.tsv"              # real data3
# save_filename_gtex = "gtex_unscaled_qntarget_batchcorrected.tsv"        # real data5
# save_filename_gtex = "gtex_unscaled_fsqn_batchcorrected.tsv"            # real data7

## Install and load packages required in R

In [ ]:
# install packages
print("  begin -- installing R packages")

options(install.packages.compile.from.source = "always")
install.packages("dplyr", repos = getCRANmirrors()[1,"URL"])
install.packages("readr", repos = getCRANmirrors()[1,"URL"])
install.packages("glue", repos = getCRANmirrors()[1,"URL"])
install.packages("stringr", repos = getCRANmirrors()[1,"URL"])
install.packages("data.table", repos = getCRANmirrors()[1,"URL"])
# install.packages("src/poibm/r/poibm/", type = "source", repos = NULL)
# install.packages("limma", repos = getCRANmirrors()[1,"URL"])
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("sva")
BiocManager::install("bladderbatch") # _TODO_forExampleCode_removeLater?
BiocManager::install("limma")

print("  done  -- installing R packages")

In [ ]:
# load packages
print("  begin -- loading packages")

library(dplyr)                  
library(tibble)                
library(readr)                  
library(limma)                  
library(stringr)                
library(data.table)             
library(sva)                    
library(bladderbatch)           

print("  done  -- loading packages")

## Indicate what indepedent dataset we will be using

In [ ]:
if (WORKING_WITH_INDEPENDENT == "GTEX"){
    print (" -----------------------------------------------------------------------")    
    print (" [NOTICE] WORKING_WITH_INDEPENDENT is GTEX")
    print (" -----------------------------------------------------------------------") 
} else if (WORKING_WITH_INDEPENDENT == "ICGC"){
    print (" -----------------------------------------------------------------------")    
    print (" [NOTICE] WORKING_WITH_INDEPENDENT is ICGC")
    print (" -----------------------------------------------------------------------") 
} else if (WORKING_WITH_INDEPENDENT == "GEO"){
    print (" -----------------------------------------------------------------------")    
    print (" [NOTICE] WORKING_WITH_INDEPENDENT is GEO")
    print (" -----------------------------------------------------------------------") 
} else if (WORKING_WITH_INDEPENDENT == "ICGCGEO"){
    print (" -----------------------------------------------------------------------")    
    print (" [NOTICE] WORKING_WITH_INDEPENDENT is ICGC and GEO combined")
    print (" -----------------------------------------------------------------------") 
} else {
    print (" -----------------------------------------------------------------------")    
    print (" [ERROR] WORKING_WITH_INDEPENDENT is not defined as expected")
    print (" -----------------------------------------------------------------------") 
}

## [FOR ICGC_GEO ONLY] need to determine cancer vs normal samples 

- info coming from Slack channel
    * (ICGC info) file called "ICGC.cancer_normal.mapped.txt" located in data/ folder
    * (GEO info) file called "GEO.SRR2labels.txt" located in data/ folder

- goal is to create a single file with two columns (sample_id, disease_type)

In [ ]:
if (WORKING_WITH_INDEPENDENT == "ICGC" || WORKING_WITH_INDEPENDENT == "ICGCGEO" ){
    
    # ---------------------- load table with readr package ----------
    filename_w_path_icgc = paste("data/metadata/", "ICGC.cancer_normal.mapped.txt", sep="")


    # (remember) the # rows total does not include header row
    sprintf("  loading ICGC metadata table -- %s", filename_w_path_icgc)
    metadata_icgc <- read_tsv(filename_w_path_icgc, col_names = FALSE)
    colnames(metadata_icgc) <- c("id_1", "id_2", "detail")
    print("  finished loading and adding header")

} 



In [ ]:
if (WORKING_WITH_INDEPENDENT == "ICGC" || WORKING_WITH_INDEPENDENT == "ICGCGEO"){
    # preview the icgc table 
    head(metadata_icgc, 2)
}

In [ ]:
if (WORKING_WITH_INDEPENDENT == "ICGC" || WORKING_WITH_INDEPENDENT == "ICGCGEO"){
    metadata_icgc_cleaned <- metadata_icgc %>% 
        mutate(sample_id = id_2) %>% 
        mutate(disease_type = case_when(
            str_detect(detail, pattern = "tumour") ~ "cancer",
            str_detect(detail, pattern = "Normal") ~ "normal",
            .default = "TODO")
        ) %>%  
        select(sample_id, disease_type)
}

In [ ]:
if (WORKING_WITH_INDEPENDENT == "ICGC" || WORKING_WITH_INDEPENDENT == "ICGCGEO"){
    # preview the top after modifying
    head(metadata_icgc_cleaned, 2)
}

In [ ]:
if (WORKING_WITH_INDEPENDENT == "ICGC" || WORKING_WITH_INDEPENDENT == "ICGCGEO"){
    print("  checking tally of disease_type of metadata_icgc_subset")
    metadata_icgc_cleaned %>%
          group_by(disease_type) %>%
          tally()
}

In [ ]:
if (WORKING_WITH_INDEPENDENT == "ICGC" || WORKING_WITH_INDEPENDENT == "ICGCGEO"){
    print("  changing type to df")
    metadata_icgc_df <- metadata_icgc_cleaned %>% 
        as.data.frame()

    print("  checking dimension")
    dim(metadata_icgc_df)
}

In [ ]:
if (WORKING_WITH_INDEPENDENT == "GEO" || WORKING_WITH_INDEPENDENT == "ICGCGEO"){
    
    # ---------------------- load table with readr package ----------
    filename_w_path_geo = paste("data/metadata/", "GEO.SRR2labels.txt", sep="")


    # (remember) the # rows total does not include header row
    sprintf("  loading GEO metadata table -- %s", filename_w_path_geo)
    metadata_geo <- read_tsv(filename_w_path_geo, col_names = FALSE)
    colnames(metadata_geo) <- c("srr_id", "gsm_id", "tissue", "detail")
    print("  finished loading and adding header")
} 

In [ ]:
if (WORKING_WITH_INDEPENDENT == "GEO" || WORKING_WITH_INDEPENDENT == "ICGCGEO"){
    # preview the top geo table 
    head(metadata_geo, 2)
}

In [ ]:
if (WORKING_WITH_INDEPENDENT == "GEO" || WORKING_WITH_INDEPENDENT == "ICGCGEO"){
    # (source for case_when) https://community.rstudio.com/t/case-when-mutate-str-detect/124835
    metadata_geo_cleaned <- metadata_geo %>% 
        mutate(sample_id = srr_id) %>% 
        mutate(disease_type = case_when(
            str_detect(detail, pattern = "tumor") ~ "cancer",
            str_detect(detail, pattern = "PVTT") ~ "cancer",
            str_detect(detail, pattern = "Cancer") ~ "cancer",
            str_detect(detail, pattern = "Normal") ~ "normal",
            str_detect(detail, pattern = "normal") ~ "normal",
            str_detect(detail, pattern = "non-malignant") ~ "normal",
            str_detect(detail, pattern = "Airway") ~ "normal",
            .default = "TODO")
        ) %>%  
        select(sample_id, disease_type)
}

In [ ]:
if (WORKING_WITH_INDEPENDENT == "GEO" || WORKING_WITH_INDEPENDENT == "ICGCGEO"){
    # preview the top after modifying
    head(metadata_geo_cleaned, 2)
}

In [ ]:
if (WORKING_WITH_INDEPENDENT == "GEO" || WORKING_WITH_INDEPENDENT == "ICGCGEO"){
    print("  checking tally of disease_type of metadata_icgc_subset")
    metadata_geo_cleaned %>%
          group_by(disease_type) %>%
          tally()
}

In [ ]:
if (WORKING_WITH_INDEPENDENT == "GEO" || WORKING_WITH_INDEPENDENT == "ICGCGEO"){
    print("  changing type to df")
    metadata_geo_df <- metadata_geo_cleaned %>% 
        as.data.frame()

    print("  checking dimension")
    dim(metadata_geo_df)
}

In [ ]:
if (WORKING_WITH_INDEPENDENT == "ICGCGEO" || WORKING_WITH_INDEPENDENT == "ICGCGEO"){
    print("  combining ICGC_GEO metadata")
    metadata_icgc_geo <- rbindlist(list(metadata_icgc_df, metadata_geo_df))

    print("  changing type to df")
    metadata_icgc_geo_df <- metadata_icgc_geo %>% 
        as.data.frame()

    print("  checking dimension")
    dim(metadata_icgc_geo)
}

## Loading up the expression data for ICGC or GEO

In [ ]:
if (WORKING_WITH_INDEPENDENT == "ICGC" || WORKING_WITH_INDEPENDENT == "GEO" || WORKING_WITH_INDEPENDENT == "ICGCGEO"){
    print("  loading ICGC or GEO dataset (will need to resave so as not to touch code down bottom)")

    filename_w_path_gtex = paste("data/preprocessing_combinations/", filename_gtex, sep="")

    if (WORKING_WITH_INDEPENDENT == "ICGC"){
        print(sprintf("  loading ICGC (but using gtex as variable name) table -- %s", filename_gtex))
    }
    if (WORKING_WITH_INDEPENDENT == "GEO"){
        print(sprintf("  loading GEO (but using gtex as variable name) table -- %s", filename_gtex))
    }
    if (WORKING_WITH_INDEPENDENT == "ICGCGEO"){
        print(sprintf("  loading ICGC and GEO combined (but using gtex as variable name) table -- %s", filename_gtex))
    }


    samples_gtex <- read_tsv(filename_w_path_gtex)
    print("  finished loading")
}

In [ ]:
if (WORKING_WITH_INDEPENDENT == "ICGC" || WORKING_WITH_INDEPENDENT == "GEO" || WORKING_WITH_INDEPENDENT == "ICGCGEO"){

    # preview the top and bottom independent dataset table 
    head(samples_gtex[,1:5], 2)
    tail(samples_gtex[,1:5], 2)
}

In [ ]:
if (WORKING_WITH_INDEPENDENT == "ICGC" || WORKING_WITH_INDEPENDENT == "GEO" || WORKING_WITH_INDEPENDENT == "ICGCGEO"){
    
    if (str_detect(samples_gtex[1,1], pattern="_cancer") || str_detect(samples_gtex[1,1], pattern="_normal")){
        print(" [NOTICE] the load ICGC_GEO sample_ids already have cancer/normal appended in name, no merge necessary")
        icgc_geo <- samples_gtex
    } else {
        
        print(" [NOTICE] the load ICGC_GEO sample_ids DO NOT have cancer/normal appended in name, thus merging")
        

        if (WORKING_WITH_INDEPENDENT == "ICGC"){
            print("  merging with ICGC dataset")
            icgc_geo <- inner_join(metadata_icgc_df, samples_gtex)
        }
        if (WORKING_WITH_INDEPENDENT == "GEO"){
            print("  merging with GEO dataset")
            icgc_geo <- inner_join(metadata_geo_df, samples_gtex)
        }
        if (WORKING_WITH_INDEPENDENT == "ICGCGEO"){
            print("  merging with ICGC_GEO dataset")
            icgc_geo <- inner_join(metadata_icgc_geo_df, samples_gtex)
        }
    }

    print("  checking dimension")
    dim(icgc_geo)
} 

In [ ]:
if (WORKING_WITH_INDEPENDENT == "ICGC" || WORKING_WITH_INDEPENDENT == "GEO" || WORKING_WITH_INDEPENDENT == "ICGCGEO"){
    print("  preview the top and bottom icgc_geo table before potential merge") 
    print(head(icgc_geo[,1:5], 2))
    print(tail(icgc_geo[,1:5], 2))
}

In [ ]:
if (WORKING_WITH_INDEPENDENT == "ICGC" || WORKING_WITH_INDEPENDENT == "GEO" || WORKING_WITH_INDEPENDENT == "ICGCGEO"){
    if (str_detect(samples_gtex[1,1], pattern="_cancer") || str_detect(samples_gtex[1,1], pattern="_normal")){
        print(" [NOTICE] the load ICGC_GEO sample_ids already have cancer/normal appended in name, no combine necessary")
    } else {
        print(" [NOTICE] combining sample_id and disease_type column into one string")
        icgc_geo <- icgc_geo %>%
            mutate(tmp_sample_id = paste(sample_id, disease_type, sep="_")) %>% 
            select(tmp_sample_id, everything(), -sample_id, -disease_type, )
    
        dim(icgc_geo)
    
        icgc_geo <- icgc_geo %>%
            mutate(sample_id = tmp_sample_id) %>% 
            select(sample_id, everything(), -tmp_sample_id, )
    }

    print("  checking dimension of icgc_geo variable before saving")
    dim(icgc_geo)
}

In [ ]:
if (WORKING_WITH_INDEPENDENT == "ICGC" || WORKING_WITH_INDEPENDENT == "GEO" || WORKING_WITH_INDEPENDENT == "ICGCGEO"){
    print("  preview the top and bottom icgc_geo table after potential merge") 
    print(head(icgc_geo[,1:5], 2))
    print(tail(icgc_geo[,1:5], 2))
}

In [ ]:
if (WORKING_WITH_INDEPENDENT == "ICGC" || WORKING_WITH_INDEPENDENT == "GEO" || WORKING_WITH_INDEPENDENT == "ICGCGEO"){
    if (str_detect(samples_gtex[1,1], pattern="_cancer") || str_detect(samples_gtex[1,1], pattern="_normal")){
        print(" [NOTICE] the load ICGC_GEO sample_ids already have cancer/normal appended in name, no save necessary")
    } else {
        print("  saving ICGC_GEO dataset to be reloaded soon (so as not to touch code down bottom)")
    
        # back to original filename
        sprintf("    writing gtex table to path -- %s", filename_w_path_gtex)
        write_tsv(icgc_geo, filename_w_path_gtex)
        print("  gtex table saved (but data inside is really either ICGC or GEO) -- ")
    }
}

## Load both datasets (TCGA and GTEx)

In [ ]:
# ---------------------- load table with readr package ----------
filename_w_path_tcga = paste("data/preprocessing_combinations/", filename_tcga, sep="")
filename_w_path_gtex = paste("data/preprocessing_combinations/", filename_gtex, sep="")


# (remember) the # rows total does not include header row
sprintf("  loading TCGA table -- %s", filename_tcga)
samples_tcga <- read_tsv(filename_w_path_tcga)
print("  finished loading")

sprintf("  loading GTEx table -- %s", filename_gtex)
samples_gtex <- read_tsv(filename_w_path_gtex)
print("  finished loading")

In [ ]:
# preview the top and bottom tcga table 
head(samples_tcga[,1:5], 2)
tail(samples_tcga[,1:5], 2)

In [ ]:
# preview the top and bottom gtex table 
head(samples_gtex[,1:5], 2)
tail(samples_gtex[,1:5], 2)

## Indicate what batch effect correction algorithm will be using

In [ ]:
if (BATCH_EFFECT_ALGORITHM_TO_APPLY == "limma"){
    print (" -----------------------------------------------------------------------")    
    print (" [NOTICE] BATCH_EFFECT_ALGORITHM_TO_APPLY equals 'limma', so will use removeBatchEffect() function")
    print (" -----------------------------------------------------------------------") 
} else if (BATCH_EFFECT_ALGORITHM_TO_APPLY == "combat"){
    print (" -----------------------------------------------------------------------")    
    print (" [NOTICE] BATCH_EFFECT_ALGORITHM_TO_APPLY equals 'combat', so will use reference-batch combat ")
    print (" -----------------------------------------------------------------------") 
} else if (BATCH_EFFECT_ALGORITHM_TO_APPLY == "poibm"){
    print (" -----------------------------------------------------------------------")    
    print (" [NOTICE] BATCH_EFFECT_ALGORITHM_TO_APPLY equals 'poibm', so will use POIBM ")
    print (" -----------------------------------------------------------------------") 
}




## Perform batch effect correction 1 of 3 (UNC vs BCGSC) on TCGA data only [Protocol-level]

In [ ]:
if (APPLY_BATCH_EFFECT_ON_PROTOCOL){
    
    print (" -----------------------------------------------------------------------")    
    print (" [NOTICE] APPLY_BATCH_EFFECT_ON_PROTOCOL is True, so removing batch effect for UNC vs BCGSC")
    print (" -----------------------------------------------------------------------") 


    print(" Begin process for -- Perform batch effect correction 1 of 3 (UNC vs BCGSC) on TCGA data only")

    sprintf("  grabbing the sample_id and labels only for later when recreating original tcga DF structure with updated values")
    samples_label <- samples_tcga %>% 
        select(sample_id, label)

    print("  adding some batch information")
    #(source on dplyr if-else) https://stackoverflow.com/questions/24459752/can-dplyr-package-be-used-for-conditional-mutating
    #(source on extending above with str_contains) https://community.rstudio.com/t/help-me-write-this-script-for-case-when-inside-dplyr-mutate-and-ill-acknowledge-by-name-you-in-my-article/6564

    # (batch codes)
    #   "BCGSC"
    #   "UNC"

    samples_w_batch <- samples_tcga %>% 
        mutate(batch = case_when (
            str_detect(label, "GI") ~ "BCGSC",
            TRUE ~ "UNC",
            )
        )  

    print("  checking tally of batch")
    samples_w_batch %>%
          group_by(batch) %>%
          tally()

    # preview the top and bottom transposed table 
    print("    (previewing this df)")
    head(samples_w_batch[,1:5], 2)
    tail(samples_w_batch[,1:5], 2)
    
} else {
    print (" -----------------------------------------------------------------------")    
    print (" [NOTICE] APPLY_BATCH_EFFECT_ON_PROTOCOL is False, so skipping")
    print (" -----------------------------------------------------------------------") 
}



In [ ]:
if (APPLY_BATCH_EFFECT_ON_PROTOCOL){

    # ** IMPORTANT ** function requires "rows correspond to probes and columns to samples" (thus must transpose)
    # (source to transpose) https://stackoverflow.com/questions/7970179/transposing-a-dataframe-maintaining-the-first-column-as-heading
    print("  preparing the 'x' variable by transposing before extracting variables for batch effect correction algorithm implemented")

    tmp_df <- samples_w_batch %>% 
        select(-sample_id, -label, -batch)
    print("    (checking dimensions of tmp_df which doesn't have sample_id nor label column)")
    print(dim(tmp_df))

    print("    (right before the transpose call - this might take awhile)")
    x = as.matrix(t(tmp_df))
    print("    (right after the transpose call; now creating header for new df ")
    colnames(x) <- samples_w_batch$sample_id

    print("    (checking dimensions of tranposed df)")
    print(dim(x))

    # preview the top and bottom transposed table 
    print("    (previewing tranposed df)")
    head(x[,1:5], 2)
    tail(x[,1:5], 2)
    
}


In [ ]:
if (APPLY_BATCH_EFFECT_ON_PROTOCOL){

    print("  preparing the 'batch' variable before batch effect correction algorithm implemented")
    tmp_df <- samples_w_batch %>% 
        select(batch)
    print("    (checking dimensions of tmp_df which is only batch info)")
    print(dim(tmp_df))

    print("    (right before the transpose call - this might take awhile)")
    batch = as.matrix(t(tmp_df))
    print("    (right after the transpose call; now creating header for new df ")
    colnames(batch) <- samples_w_batch$sample_id

    print("    (checking dimensions of tranposed df)")
    print(dim(batch))

    # preview the top and bottom transposed table 
    print("    (previewing tranposed df)")
    head(batch[,1:5], 2)
    tail(batch[,1:5], 2)
    
}

In [ ]:
if (APPLY_BATCH_EFFECT_ON_PROTOCOL){
    print("    (checking dimensions of expression data before batch effect correction algorithm implemented)")
    print(dim(x))
    print("    (previewing the expression data before batch effect correction algorithm implemented)")
    head(x[,1:5], 2)
}

In [ ]:
if (APPLY_BATCH_EFFECT_ON_PROTOCOL){

    
    if (BATCH_EFFECT_ALGORITHM_TO_APPLY == "limma"){
        print("  performing limma's removeBatchEffect function")
        # (source) https://rdrr.io/bioc/limma/man/removeBatchEffect.html

        samples_tcga <- removeBatchEffect(x = x, batch = batch)
        
    } else if (BATCH_EFFECT_ALGORITHM_TO_APPLY == "combat"){
        print("  performing reference-batch ComBat")
        # (source for combat) https://bioconductor.org/packages/release/bioc/html/sva.html
        # (source for a paper that uses ref_batch) https://github.com/zhangyuqing/meanonly_reference_combat/blob/master/generate_figures_refCombat.R

        phenocombat <- samples_label %>%
           column_to_rownames(var = "sample_id")

        modcombat = model.matrix(~1, data=phenocombat)

        batchcombat = unname(unlist(batch[1,]))

#         ## (algorithm #A) use combat with    ref.batch with reference batches ('UNC', 'normal', 'tcga') ref
#         combat_edata = ComBat(dat = x, batch = batchcombat, mod = modcombat, par.prior=TRUE, prior.plot=FALSE, ref.batch="UNC")

#         ## (algorithm #B) use combat with    ref.batch with reference batches ('UNC', 'cancer', 'tcga') refcancer
        combat_edata = ComBat(dat = x, batch = batchcombat, mod = modcombat, par.prior=TRUE, prior.plot=FALSE, ref.batch="UNC")
        
#         ## (algorithm #C) use combat without ref.batch (i.e. default combat via sva package) refdefault
#         combat_edata = ComBat(dat = x, batch = batchcombat, mod = modcombat, par.prior=TRUE, prior.plot=FALSE)

        samples_tcga <- combat_edata
    } else if (BATCH_EFFECT_ALGORITHM_TO_APPLY == "poibm"){
        print("  performing POIBM")
        # (source) https://bitbucket.org/anthakki/poibm/src/master/

        print("    generating the x_with_batch variable")
        x_with_batch <- rbindlist(list(batch, x))
        x_with_batch <- x_with_batch %>% 
            as.data.frame()
        rownames(x_with_batch) <- c("batch", rownames(x))

        
        print("    generating the x_for_poibm variable")
        x_for_poibm <- x_with_batch %>% 
            select(where(~any(. == "UNC"))) %>%
            filter(!row_number() %in% c(1)) %>%
            mutate_if(is.character, as.numeric) %>%
            as.matrix()
        
        print("    generating the y_for_poibm variable")
        y_for_poibm <- x_with_batch %>% 
            select(where(~any(. == "BCGSC"))) %>%
            filter(!row_number() %in% c(1)) %>%
            mutate_if(is.character, as.numeric) %>%
            as.matrix()
        
        print("==========================================")
        current_date_time <- format(Sys.time(), "%Y-%m-%d %H:%M:%S")
        print(sprintf("[TIMESTAMP] %s", current_date_time))
        print("==========================================")
        
        print("    making the poibm package fit() call to get y_corrected matrix")        
        print("      the batch correction of Y into the space of X")
        fit <- poibm.fit(x_for_poibm, y_for_poibm, max.iter=20, max.resets=20, seed=12345, verbose=TRUE)
        y_corrected_for_poibm <- poibm.apply(y_for_poibm, fit)

        print("==========================================")
        current_date_time <- format(Sys.time(), "%Y-%m-%d %H:%M:%S")
        print(sprintf("[TIMESTAMP] %s", current_date_time))
        print("==========================================")
        
        print("    generating the updated x variable to reassign into the expression data variable")
        matrix_after_poibm <- cbind(x_for_poibm, y_corrected_for_poibm)  
        
        samples_tcga <- matrix_after_poibm
    }


    
    
    print("    (checking dimensions of expression data after batch effect correction algorithm implemented)")
    print(dim(samples_tcga))
    print("    (previewing the expression data after batch effect correction algorithm implemented)")
    head(samples_tcga[,1:5], 2)

}

## After done with 1st batch effect removals, need to recreate original tcga DF

In [ ]:
if (APPLY_BATCH_EFFECT_ON_PROTOCOL){

    print("  Recreating the original tcga df with updated expression values post batch effect")

    tmp_df <- samples_tcga
    print("    (checking dimensions of tmp_df)")
    print(dim(tmp_df))

    print("    (right before the transpose call) this might take awhile)")
    tmp_df = data.frame(t(tmp_df))

    print("    (right after the transpose call) now moving row names to a column ")
    tmp_df$sample_id <- rownames(tmp_df)

    print("    (need to) perform an outer join with earlier df that contains label column")
    tmp_df <- merge(x=tmp_df, y=samples_label, by="sample_id", all=TRUE)

    print("     (reorganize df column order)")
    updated_samples <- tmp_df %>% 
        select(sample_id, label, everything())

    print("    (checking dimensions of tranposed df)")
    print(dim(updated_samples))

    # preview the top and bottom transposed table 
    print("    (previewing tranposed df)")
    head(updated_samples[,1:5], 2)
    tail(updated_samples[,1:5], 2)

    samples_tcga = updated_samples
    
}

## Combine tcga and gtex


In [ ]:
sprintf("  combining tcga and gtex datasets")
samples <- rbindlist(list(samples_tcga, samples_gtex))

sprintf("  changing type to df")
samples <- samples %>% 
    as.data.frame()

sprintf("  checking dimensions of each individual and combined datasets")
print("   (combined samples)")
print(dim(samples))
print("   (tcga samples)")
print(dim(samples_tcga))
print("   (gtex samples)")
print(dim(samples_gtex))

In [ ]:
# preview the top and bottom tcga table 
head(samples[,1:5], 2)
tail(samples[,1:5], 2)

## Perform batch effect correction 2 of 3 (cancer vs normal) on both datasets  [Disease-level]

In [ ]:
if (APPLY_BATCH_EFFECT_ON_DISEASE){
    
    print (" -----------------------------------------------------------------------")    
    print (" [NOTICE] APPLY_BATCH_EFFECT_ON_DISEASE is True, so removing batch effect for cancer vs normal")
    print (" -----------------------------------------------------------------------") 



    print(" Begin process for -- Perform batch effect correction 2 of 3 (cancer vs normal) on both datasets")

    sprintf("  grabbing the sample_id and labels only for later when recreating original DF structure with updated values")
    samples_label <- samples %>% 
        select(sample_id, label)

    print("  adding some batch information")
    #(source on dplyr if-else) https://stackoverflow.com/questions/24459752/can-dplyr-package-be-used-for-conditional-mutating
    #(source on extending above with str_contains) https://community.rstudio.com/t/help-me-write-this-script-for-case-when-inside-dplyr-mutate-and-ill-acknowledge-by-name-you-in-my-article/6564

    # (batch codes)
    #   "normal"
    #   "cancer"

    samples_w_batch <- samples %>% 
        mutate(batch = case_when (
            str_detect(sample_id, "-11A") ~ "normal",    # for tcga
            str_detect(sample_id, "-01A") ~ "cancer",    # for tcga
            str_detect(sample_id, "GTEX") ~ "normal",    # for gtex
            str_detect(sample_id, "normal") ~ "normal",  # for icgc_geo
            str_detect(sample_id, "cancer") ~ "cancer",  # for icgc_geo
            TRUE ~ "TODO",                             
            )
        )  

    print("  checking tally of batch")
    samples_w_batch %>%
          group_by(batch) %>%
          tally()

    # preview the top and bottom transposed table 
    print("    (previewing this df)")
    head(samples_w_batch[,1:5], 2)
    tail(samples_w_batch[,1:5], 2)
    
} else {
    print (" -----------------------------------------------------------------------")    
    print (" [NOTICE] APPLY_BATCH_EFFECT_ON_DISEASE is False, so skipping")
    print (" -----------------------------------------------------------------------") 
}


In [ ]:
if (APPLY_BATCH_EFFECT_ON_DISEASE){

    # ** IMPORTANT ** function requires "rows correspond to probes and columns to samples" (thus must transpose)
    # (source to transpose) https://stackoverflow.com/questions/7970179/transposing-a-dataframe-maintaining-the-first-column-as-heading
    print("  preparing the 'x' variable by transposing before extracting variables for batch effect correction algorithm implemented")

    tmp_df <- samples_w_batch %>% 
        select(-sample_id, -label, -batch)
    print("    (checking dimensions of tmp_df which doesn't have sample_id nor label column)")
    print(dim(tmp_df))

    print("    (right before the transpose call - this might take awhile)")
    x = as.matrix(t(tmp_df))
    print("    (right after the transpose call; now creating header for new df ")
    colnames(x) <- samples_w_batch$sample_id

    print("    (checking dimensions of tranposed df)")
    print(dim(x))

    # preview the top and bottom transposed table 
    print("    (previewing tranposed df)")
    head(x[,1:5], 2)
    tail(x[,1:5], 2)

}


In [ ]:
if (APPLY_BATCH_EFFECT_ON_DISEASE){

    print("  preparing the 'batch' variable before batch effect correction algorithm implemented")
    tmp_df <- samples_w_batch %>% 
        select(batch)
    print("    (checking dimensions of tmp_df which is only batch info)")
    print(dim(tmp_df))

    print("    (right before the transpose call - this might take awhile)")
    batch = as.matrix(t(tmp_df))
    print("    (right after the transpose call; now creating header for new df ")
    colnames(batch) <- samples_w_batch$sample_id

    print("    (checking dimensions of tranposed df)")
    print(dim(batch))

    # preview the top and bottom transposed table 
    print("    (previewing tranposed df)")
    head(batch[,1:5], 2)
    tail(batch[,1:5], 2)

}

In [ ]:
if (APPLY_BATCH_EFFECT_ON_DISEASE){
    print("    (checking dimensions of expression data before batch effect correction algorithm implemented)")
    print(dim(x))
    print("    (previewing the expression data before batch effect correction algorithm implemented)")
    head(x[,1:5], 2)
}

In [ ]:
if (APPLY_BATCH_EFFECT_ON_DISEASE){

    if (BATCH_EFFECT_ALGORITHM_TO_APPLY == "limma"){
        print("  performing limma's removeBatchEffect function")
        # (source) https://rdrr.io/bioc/limma/man/removeBatchEffect.html
        
        samples <- removeBatchEffect(x = x, batch = batch)
        
    } else if (BATCH_EFFECT_ALGORITHM_TO_APPLY == "combat"){
        print("  performing reference-batch ComBat")

        phenocombat <- samples_label %>%
           column_to_rownames(var = "sample_id")

        modcombat = model.matrix(~1, data=phenocombat)

        batchcombat = unname(unlist(batch[1,]))

        combat_edata = ComBat(dat = x, batch = batchcombat, mod = modcombat, par.prior=TRUE, prior.plot=FALSE, ref.batch="cancer")

#         ## (algorithm #A) use combat with    ref.batch with reference batches ('UNC', 'normal', 'tcga') ref
#         combat_edata = ComBat(dat = x, batch = batchcombat, mod = modcombat, par.prior=TRUE, prior.plot=FALSE, ref.batch="normal")

#         ## (algorithm #B) use combat with    ref.batch with reference batches ('UNC', 'cancer', 'tcga') refcancer
        combat_edata = ComBat(dat = x, batch = batchcombat, mod = modcombat, par.prior=TRUE, prior.plot=FALSE, ref.batch="cancer")
        
#         ## (algorithm #C) use combat without ref.batch (i.e. default combat via sva package) refdefault
#         combat_edata = ComBat(dat = x, batch = batchcombat, mod = modcombat, par.prior=TRUE, prior.plot=FALSE)
       
        samples <- combat_edata
    } else if (BATCH_EFFECT_ALGORITHM_TO_APPLY == "poibm"){
        print("  performing POIBM")
        # (source) https://bitbucket.org/anthakki/poibm/src/master/

        print("    generating the x_with_batch variable")
        x_with_batch <- rbindlist(list(batch, x))
        x_with_batch <- x_with_batch %>% 
            as.data.frame()
        rownames(x_with_batch) <- c("batch", rownames(x))

        
        print("    generating the x_for_poibm variable")
        x_for_poibm <- x_with_batch %>% 
            select(where(~any(. == "cancer"))) %>%
            filter(!row_number() %in% c(1)) %>%
            mutate_if(is.character, as.numeric) %>%
            as.matrix()
        
        print("    generating the y_for_poibm variable")
        y_for_poibm <- x_with_batch %>% 
            select(where(~any(. == "normal"))) %>%
            filter(!row_number() %in% c(1)) %>%
            mutate_if(is.character, as.numeric) %>%
            as.matrix()
        

        print("==========================================")
        current_date_time <- format(Sys.time(), "%Y-%m-%d %H:%M:%S")
        print(sprintf("[TIMESTAMP] %s", current_date_time))
        print("==========================================")
        
        print("    making the poibm package fit() call to get y_corrected matrix")        
        print("      the batch correction of Y into the space of X")
        fit <- poibm.fit(x_for_poibm, y_for_poibm, max.iter=20, max.resets=20, seed=12345, verbose=TRUE)
        y_corrected_for_poibm <- poibm.apply(y_for_poibm, fit)

        print("==========================================")
        current_date_time <- format(Sys.time(), "%Y-%m-%d %H:%M:%S")
        print(sprintf("[TIMESTAMP] %s", current_date_time))
        print("==========================================")

        
        print("    generating the updated x variable to reassign into the expression data variable")
        matrix_after_poibm <- cbind(x_for_poibm, y_corrected_for_poibm)  
        
        samples <- matrix_after_poibm
    }

    print("    (checking dimensions of expression data after batch effect correction algorithm implemented)")
    print(dim(samples))
    print("    (previewing the expression data  after batch effect correction algorithm implemented)")
    head(samples[,1:5], 2)
    
}

## After done with 2nd batch effect removals, need to recreate original combined DF

In [ ]:
if (APPLY_BATCH_EFFECT_ON_DISEASE){

    print("  Recreating the original df with updated expression values post batch effect")

    tmp_df <- samples
    print("    (checking dimensions of tmp_df)")
    print(dim(tmp_df))

    print("    (right before the transpose call) this might take awhile)")
    tmp_df = data.frame(t(tmp_df))

    print("    (right after the transpose call) now moving row names to a column ")
    tmp_df$sample_id <- rownames(tmp_df)

    print("    (need to) perform an outer join with earlier df that contains label column")
    tmp_df <- merge(x=tmp_df, y=samples_label, by="sample_id", all=TRUE)

    print("     (reorganize df column order)")
    updated_samples <- tmp_df %>% 
        select(sample_id, label, everything())

    print("    (checking dimensions of tranposed df)")
    print(dim(updated_samples))

    # preview the top and bottom transposed table 
    print("    (previewing tranposed df)")
    head(updated_samples[,1:5], 2)
    tail(updated_samples[,1:5], 2)

    samples = updated_samples
    
}

## Perform batch effect correction 3 of 3 (TCGA vs GTEx) on both datasets  [Consortium-level]

In [ ]:
if (APPLY_BATCH_EFFECT_ON_CONSORTIUM){
    
    print (" -----------------------------------------------------------------------")    
    print (" [NOTICE] APPLY_BATCH_EFFECT_ON_CONSORTIUM is True, so removing batch effect for TCGA vs GTEX")
    print (" -----------------------------------------------------------------------") 


    print(" Perform batch effect correction 3 of 3 (TCGA vs GTEx) on both datasets")

    sprintf("  grabbing the sample_id and labels only for later when recreating original tcga DF structure with updated values")
    samples_label <- samples %>% 
    select(sample_id, label)
    
    print("  adding some batch information")
    #(source on dplyr if-else) https://stackoverflow.com/questions/24459752/can-dplyr-package-be-used-for-conditional-mutating
    #(source on extending above with str_contains) https://community.rstudio.com/t/help-me-write-this-script-for-case-when-inside-dplyr-mutate-and-ill-acknowledge-by-name-you-in-my-article/6564

    # (batch codes)
    #   "tcga"
    #   "gtex"

    samples_w_batch <- samples %>% 
        mutate(batch = case_when (
            str_detect(sample_id, "-11A") ~ "tcga",
            str_detect(sample_id, "-01A") ~ "tcga",
            str_detect(sample_id, "GTEX") ~ "gtex",
            TRUE ~ "icgc",
            )
        )  

    print("  checking tally of batch")
    samples_w_batch %>%
          group_by(batch) %>%
          tally()

    # preview the top and bottom transposed table 
    print("    (previewing this df)")
    head(samples_w_batch[,1:5], 2)
    tail(samples_w_batch[,1:5], 2)

    
} else {
    print (" -----------------------------------------------------------------------")    
    print (" [NOTICE] APPLY_BATCH_EFFECT_ON_CONSORTIUM is False, so skipping")
    print (" -----------------------------------------------------------------------") 
}

In [ ]:
if (APPLY_BATCH_EFFECT_ON_CONSORTIUM){

    # ** IMPORTANT ** function requires "rows correspond to probes and columns to samples" (thus must transpose)
    # (source to transpose) https://stackoverflow.com/questions/7970179/transposing-a-dataframe-maintaining-the-first-column-as-heading
    print("  preparing the 'x' variable by transposing before extracting variables for batch effect correction algorithm implemented")

    tmp_df <- samples_w_batch %>% 
        select(-sample_id, -label, -batch)
    print("    (checking dimensions of tmp_df which doesn't have sample_id nor label column)")
    print(dim(tmp_df))

    print("    (right before the transpose call - this might take awhile)")
    x = as.matrix(t(tmp_df))
    print("    (right after the transpose call; now creating header for new df ")
    colnames(x) <- samples_w_batch$sample_id

    print("    (checking dimensions of tranposed df)")
    print(dim(x))

    # preview the top and bottom transposed table 
    print("    (previewing tranposed df)")
    head(x[,1:5], 2)
    tail(x[,1:5], 2)
}

In [ ]:
if (APPLY_BATCH_EFFECT_ON_CONSORTIUM){

    print("  preparing the 'batch' variable before batch effect correction algorithm implemented")
    tmp_df <- samples_w_batch %>% 
        select(batch)
    print("    (checking dimensions of tmp_df which is only batch info)")
    print(dim(tmp_df))

    print("    (right before the transpose call - this might take awhile)")
    batch = as.matrix(t(tmp_df))
    print("    (right after the transpose call; now creating header for new df ")
    colnames(batch) <- samples_w_batch$sample_id

    print("    (checking dimensions of tranposed df)")
    print(dim(batch))

    # preview the top and bottom transposed table 
    print("    (previewing tranposed df)")
    head(batch[,1:5], 2)
    tail(batch[,1:5], 2)
    
}

In [ ]:
if (APPLY_BATCH_EFFECT_ON_CONSORTIUM){
    print("    (checking dimensions of expression data before batch effect correction algorithm implemented)")
    print(dim(x))
    print("    (previewing the expression data before batch effect correction algorithm implemented)")
    head(x[,1:5], 2)
}

In [ ]:
if (APPLY_BATCH_EFFECT_ON_CONSORTIUM){

    if (BATCH_EFFECT_ALGORITHM_TO_APPLY == "limma"){
        print("  performing limma's removeBatchEffect function")
        # (source) https://rdrr.io/bioc/limma/man/removeBatchEffect.html
        
        samples <- removeBatchEffect(x = x, batch = batch)
        
    } else if (BATCH_EFFECT_ALGORITHM_TO_APPLY == "combat"){
        print("  performing reference-batch ComBat")

        phenocombat <- samples_label %>%
           column_to_rownames(var = "sample_id")

        modcombat = model.matrix(~1, data=phenocombat)

        batchcombat = unname(unlist(batch[1,]))

#         ## (algorithm #A) use combat with    ref.batch with reference batches ('UNC', 'normal', 'tcga') ref
#         combat_edata = ComBat(dat = x, batch = batchcombat, mod = modcombat, par.prior=TRUE, prior.plot=FALSE, ref.batch="tcga")

#         ## (algorithm #B) use combat with    ref.batch with reference batches ('UNC', 'cancer', 'tcga') refcancer
        combat_edata = ComBat(dat = x, batch = batchcombat, mod = modcombat, par.prior=TRUE, prior.plot=FALSE, ref.batch="tcga")
        
#         ## (algorithm #C) use combat without ref.batch (i.e. default combat via sva package) refdefault
#         combat_edata = ComBat(dat = x, batch = batchcombat, mod = modcombat, par.prior=TRUE, prior.plot=FALSE)
        
        
        
        
        samples <- combat_edata
    } else if (BATCH_EFFECT_ALGORITHM_TO_APPLY == "poibm"){
        print("  performing POIBM")
        # (source) https://bitbucket.org/anthakki/poibm/src/master/

        print("    generating the x_with_batch variable")
        x_with_batch <- rbindlist(list(batch, x))
        x_with_batch <- x_with_batch %>% 
            as.data.frame()
        rownames(x_with_batch) <- c("batch", rownames(x))

        
        print("    generating the x_for_poibm variable")
        x_for_poibm <- x_with_batch %>% 
            select(where(~any(. == "tcga"))) %>%
            filter(!row_number() %in% c(1)) %>%
            mutate_if(is.character, as.numeric) %>%
            as.matrix()
        
        print("    generating the y_for_poibm variable")
        y_for_poibm <- x_with_batch %>% 
            select(where(~any(. == "gtex"))) %>%
            filter(!row_number() %in% c(1)) %>%
            mutate_if(is.character, as.numeric) %>%
            as.matrix()
        

        print("==========================================")
        current_date_time <- format(Sys.time(), "%Y-%m-%d %H:%M:%S")
        print(sprintf("[TIMESTAMP] %s", current_date_time))
        print("==========================================")
        
        print("    making the poibm package fit() call to get y_corrected matrix")        
        print("      the batch correction of Y into the space of X")
        fit <- poibm.fit(x_for_poibm, y_for_poibm, max.iter=20, max.resets=20, seed=12345, verbose=TRUE)
        y_corrected_for_poibm <- poibm.apply(y_for_poibm, fit)

        print("==========================================")
        current_date_time <- format(Sys.time(), "%Y-%m-%d %H:%M:%S")
        print(sprintf("[TIMESTAMP] %s", current_date_time))
        print("==========================================")

        
        print("    generating the updated x variable to reassign into the expression data variable")
        matrix_after_poibm <- cbind(x_for_poibm, y_corrected_for_poibm)  
        
        samples <- matrix_after_poibm
    }

    
    
    print("    (checking dimensions of expression data after batch effect correction algorithm implemented)")
    print(dim(samples))
    print("    (previewing the expression data after batch effect correction algorithm implemented)")
    head(samples[,1:5], 2)
    
}

## After done with batch effect removal #3, need to recreate original DF

In [ ]:
if (APPLY_BATCH_EFFECT_ON_CONSORTIUM){

    print("  Recreating the original df with updated expression values post batch effect")

    tmp_df <- samples
    print("    (checking dimensions of tmp_df)")
    print(dim(tmp_df))

    print("    (right before the transpose call) this might take awhile)")
    tmp_df = data.frame(t(tmp_df))

    print("    (right after the transpose call) now moving row names to a column ")
    tmp_df$sample_id <- rownames(tmp_df)

    print("    (need to) perform an outer join with earlier df that contains label column")
    tmp_df <- merge(x=tmp_df, y=samples_label, by="sample_id", all=TRUE)

    print("     (reorganize df column order)")
    samples <- tmp_df %>% 
        select(sample_id, label, everything())

    print("    (checking dimensions of tranposed df)")
    print(dim(samples))

    # preview the top and bottom transposed table 
    print("    (previewing tranposed df)")
    head(samples[,1:5], 2)
    tail(samples[,1:5], 2)
}

## Split up the updated dataset into TCGA and GTEx

In [ ]:
print ("  Subsetting out tcga samples ")

updated_samples_tcga <- samples %>% 
    filter(grepl("-11A|-01A",sample_id))

print("  sort by label then by sample_id")
updated_samples_tcga <- updated_samples_tcga %>%
   group_by(label) %>%
   arrange(sample_id, .by_group = TRUE)

print ("    check dimensions of final tcga dataset")
dim(updated_samples_tcga)

print ("    preview of final tcga dataset")
print("    (previewing tranposed df)")
head(updated_samples_tcga[,1:5], 2)
tail(updated_samples_tcga[,1:5], 2)

In [ ]:
print ("  Subsetting out gtex samples ")

updated_samples_gtex <- samples %>% 
    filter(!grepl("-11A|-01A",sample_id))

# print("  sort by label then by sample_id")
# updated_samples_gtex <- updated_samples_gtex %>%
#    group_by(label) %>%
#    arrange(sample_id, .by_group = TRUE)

print ("    check dimensions of final gtex dataset")
dim(updated_samples_gtex)

print ("    preview of final gtex dataset")
print("    (previewing tranposed df)")
head(updated_samples_gtex[,1:5], 2)
tail(updated_samples_gtex[,1:5], 2)

## Split tcga into 80% train and 20% test; then recombine right away

- need to have the order of samples ready for ML (first 80% of samples are train; rest for test)

In [ ]:
set.seed(12345)            # IMPORTANT (this should be the same seed as used in notebook08)

print("  sort tcga by label then by sample_id")
updated_samples_tcga <- updated_samples_tcga %>%
   group_by(label) %>%
   arrange(sample_id, .by_group = TRUE)

print("  perform the actual train-test split at 80%")
samples_tcga_train <- updated_samples_tcga  %>% 
    sample_frac(0.80)
samples_tcga_test <- anti_join(updated_samples_tcga, samples_tcga_train, by = "sample_id" )

print("  sort tcga_train by label then by sample_id")
samples_tcga_train <- samples_tcga_train %>%
   group_by(label) %>%
   arrange(sample_id, .by_group = TRUE)

print("  sort tcga_test by label then by sample_id")
samples_tcga_test <- samples_tcga_test %>%
   group_by(label) %>%
   arrange(sample_id, .by_group = TRUE)

print("  checking dimensions and tally of 80% TCGA for train set")
dim(samples_tcga_train)
samples_tcga_train %>%
      group_by(label) %>%
      tally()

print("  checking dimensions and tally of 20% TCGA for test set")
dim(samples_tcga_test)
samples_tcga_test %>%
      group_by(label) %>%
      tally()

In [ ]:
print ("previewing the top and bottom 80% tcga table") 
print (head(samples_tcga_train[,1:5], 2) )
print (tail(samples_tcga_train[,1:5], 2) )

In [ ]:
print ("previewing the top and bottom 20% tcga table") 
print (head(samples_tcga_test[,1:5], 2) )
print (tail(samples_tcga_test[,1:5], 2) )

In [ ]:
print("  combining train set (80% TCGA) and test set (20% TCGA)")
samples_tcga <- rbindlist(list(
                                    samples_tcga_train, 
                                    samples_tcga_test
                                    ))

print("  checking dimensions of each individual and combined datasets")
print("   (100% TCGA samples)")
print(dim(samples_tcga))
print("   (80%  TCGA samples)")
print(dim(samples_tcga_train))
print("   (20%  TCGA samples)")
print(dim(samples_tcga_test))

## Save the files

In [ ]:
print ("  Saving the files ")

# ---------------------- TCGA dataset ---------------------------

save_filename_w_path_tcga = paste("data/preprocessing_combinations/", save_filename_tcga, sep="")

sprintf("    writing tcga table to path -- %s", save_filename_w_path_tcga)
write_tsv(samples_tcga, save_filename_w_path_tcga)
print("  tcga table saved-- ")

# ---------------------- GTEx dataset ---------------------------

save_filename_w_path_gtex = paste("data/preprocessing_combinations/", save_filename_gtex, sep="")

sprintf("    writing gtex table to path -- %s", save_filename_w_path_gtex)
write_tsv(updated_samples_gtex, save_filename_w_path_gtex)
print("  gtex table saved-- ")


